In [37]:
import numpy as np
from Bio import Phylo
from scipy.special import comb
from utilities import *

np.random.seed(42)

In [38]:
class GenSimulator():
    """
    Base class that provide simulation for one genom
    """
    def __init__(self, number_of_descendants: int, 
                 population_size_over_time: int,
                 mutation_rate: int,
                 lambda_distributation):
        self.descendants = number_of_descendants
        self.lambda_distributation = lambda_distributation
        self.population_size_over_time = population_size_over_time
        self.mutation_rate = mutation_rate
        
        self.poisson_rate = number_of_descendants*mutation_rate + comb(number_of_descendants,2)/lambda_distributation(0)
        
        self.current_population = {}
        self.tree = None
        self.table = None
    
    
    def init_population(self):
        for i in range(self.descendants):
            name = generate_name()
            while name in self.current_population.keys(): name = generate_name()
            self.current_population[name] = "{}:{}".format(name, '{}')
            
    def do_event(self, event_type, *args):
        if event_type == 'coalescence':
            self.do_coalescence(*args)
        elif event_type = 'mutation':
            self.do_mutation(*args)
        else:
            raise 'Type not in list'
    
    def do_coalescence(self, current_linage_number, t, delta_t):
        name_1 = np.random.choice(self.current_population)
        name_2 = np.random.choice(self.current_population)
        while name_1 = name_2: name_2 = np.random.choice(self.current_population)
        left = self.current_population.pop(name_1)
        right = self.current_population.pop(name_2)
        new_name = '{}_{}'.format(name_1, name_2)
        new_branch = '(' + left + ',' + right + ')'
        self.current_population[new_name] =  new_branch.format(delta_t)
        self.poisson_rate = (current_linage_number-1)*mutation_rate + comb(current_linage_number-1,2)/lambda_distributation(t)
    
    def do_mutation(self, current_linage_number, t, delta_t):
        position = np.random.uniform()
        while position in self.table.keys: position = np.random.uniform()
        name = np.random.choice(self.current_population)
        self.table[position] = name
        
    def generate(self):
        t = 0
        i = 0
        current_limit = self.population_size_over_time[i]
        current_linage_number = self.number_of_descendants
        
        while current_linage_number > 1:
            generated_t = np.random.poisson(self.poisson_rate)
            if generated_t > current_limit:
                # Nothing happens in this intervals
                i = i + 1
                t = current_limit
                current_limit = self.population_size_over_time[i]
                self.poisson_rate = current_linage_number*mutation_rate + comb(current_linage_number,2)/lambda_distributation(t)
                continue
            else:
                t = t + generated_t
                # Chose what happens - mutation or coalesent
                event_type = np.random.choice(
                    ['coalescence', 'mutation'],
                    p=[current_linage_number*mutation_rate,
                       comb(current_linage_number,2)/lambda_distributation(t)]
                )
                self.do_event(event_type, current_linage_number, t, generated_t)
                current_linage_number = len(self.current_population)
        for tree in self.current_population.values():
            self.tree = tree
    
    def __call__(self):
        if self.tree is None or self.table is None:
            raise "Firstly need to be generated"
        return self.tree, self.table
    

SyntaxError: invalid syntax (<ipython-input-38-845a116f40fe>, line 23)

In [53]:
a = {}

In [54]:
name = 'q'
if name not in a.keys():
    a[name] = 1
a

{'q': 1}

In [50]:
'{}:{}'.format('a', '{}')

'a:{}'

In [51]:
a.pop('q')

1

In [64]:
for d in a.values(): print(d)

1
